In [3]:
# conda activate anndata

import re
import os
import sys
import numpy as np
import pandas as pd
import anndata as ad
from scipy.io import mmwrite
from functools import reduce
from scipy.sparse import csr_matrix

sys.path.append("/mnt/lareaulab/reliscu/code")

from junction2psi import *

## Metadata

In [ ]:
sampleinfo = pd.read_csv("../GTEx_Analysis_v10_Annotations_SampleAttributesDS.txt", sep="\t")
donorinfo = pd.read_csv("../GTEx_Analysis_v10_Annotations_SubjectPhenotypesDS.txt", sep="\t")

/tmp/ipykernel_2788871/2053469459.py:1: DtypeWarning: Columns (18) have mixed types. Specify dtype option on import or set low_memory=False.
  sampleinfo = pd.read_csv("../GTEx_Analysis_v10_Annotations_SampleAttributesDS.txt", sep="\t")


In [5]:
sampleinfo['Donor'] = sampleinfo['SAMPID'].str.split("-").str[0:2].str.join("-")

sampleinfo_FCX = sampleinfo.loc[sampleinfo['SMTSD'].str.contains("Frontal Cortex") & sampleinfo['SMAFRZE'].str.contains("RNASEQ", na=False)]
sampleinfo_FCX = sampleinfo_FCX.merge(donorinfo, left_on="Donor", right_on="SUBJID", how="left")

In [6]:
sampleinfo_FCX.shape

(269, 124)

In [7]:
# sampleinfo_FCX.to_csv("GTEx_frontal_cortex_sampleinfo.csv", index=False)

## Gene expression

In [7]:
expr_tpm = pd.read_csv("../GTEx_Analysis_v10_RNASeQCv2.4.2_gene_tpm.gct", sep="\t", header=2)

In [8]:
expr_tpm.head()

,Name,Description,GTEX-1117F-0005-SM-HL9SH,GTEX-1117F-0011-R10b-SM-GI4VE,GTEX-1117F-0011-R11b-SM-GIN8R,GTEX-1117F-0011-R2b-SM-GI4VL,GTEX-1117F-0011-R3a-SM-GJ3PJ,GTEX-1117F-0011-R4b-SM-GI4VM,GTEX-1117F-0011-R5a-SM-GI4VW,GTEX-1117F-0011-R6a-SM-GI4VX,...,GTEX-ZZPU-1326-SM-5GZWS,GTEX-ZZPU-1426-SM-5GZZ6,GTEX-ZZPU-1826-SM-5E43L,GTEX-ZZPU-2126-SM-5EGIU,GTEX-ZZPU-2226-SM-5EGIV,GTEX-ZZPU-2326-SM-GOQYU,GTEX-ZZPU-2426-SM-5E44I,GTEX-ZZPU-2526-SM-GOQZ3,GTEX-ZZPU-2626-SM-5E45Y,GTEX-ZZPU-2726-SM-5NQ8O
0,ENSG00000223972.5,DDX11L1,0.00000,0.000000,0.000000,0.00000,0.0000,0.012397,0.00000,0.029395,...,0.000000,0.0000,0.000000,0.00000,0.00000,0.000000,0.000000,0.017791,0.020221,0.025713
1,ENSG00000227232.5,WASH7P,1.33343,3.579280,10.189300,2.96650,3.6828,3.599210,3.51443,4.286380,...,5.464380,2.3599,2.427010,3.87837,1.85892,2.383760,2.886100,5.095810,0.842465,2.209520
2,ENSG00000278267.1,MIR6859-1,0.00000,0.000000,0.000000,0.00000,0.0000,0.000000,0.00000,0.000000,...,0.000000,0.0000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
3,ENSG00000243485.5,MIR1302-2HG,0.00000,0.093825,0.034191,0.00000,0.0000,0.024750,0.00000,0.000000,...,0.062071,0.0000,0.086553,0.14685,0.00000,0.041073,0.053323,0.000000,0.000000,0.000000
4,ENSG00000237613.2,FAM138A,0.00000,0.000000,0.000000,0.01766,0.0000,0.000000,0.00000,0.000000,...,0.000000,0.0000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000


In [51]:
expr_tpm = expr_tpm.set_index("Description", drop=True)

In [60]:
expr_tpm_FCX = expr_tpm.loc[:, sampleinfo_FCX['SAMPID']]

In [ ]:
expr_tpm_FCX.head()

,GTEX-1117F-0011-R10b-SM-GI4VE,GTEX-111FC-0011-R10a-SM-GIN8G,GTEX-117XS-0011-R10b-SM-GIN8Z,GTEX-1192W-0011-R10b-SM-GHWOF,GTEX-1192X-0011-R10a-SM-DO941,GTEX-11DXW-0011-R10b-SM-GI4VP,GTEX-11DXY-0011-R10b-SM-DO12C,GTEX-11DYG-0011-R10b-SM-DNZZO,GTEX-11DZ1-0011-R10b-SM-DO943,GTEX-11EI6-0011-R10a-SM-DO93R,...,GTEX-ZE7O-0011-R10a-SM-57WAZ,GTEX-ZF28-0011-R10a-SM-4WWEH,GTEX-ZUA1-0011-R10a-SM-51MT6,GTEX-ZV68-0011-R10a-SM-51MT7,GTEX-ZVT3-0011-R10b-SM-57WB6,GTEX-ZVZQ-0011-R10b-SM-51MRT,GTEX-ZXG5-0011-R10a-SM-57WDD,GTEX-ZYFD-0011-R10a-SM-GPI91,GTEX-ZYY3-0011-R10a-SM-GNTAZ,GTEX-ZZPT-0011-R10b-SM-GPI8B
Description,,,,,,,,,,,,,,,,,,,,,
DDX11L1,0.000000,0.000000,0.000000,0.000000,0.040388,0.017816,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.018103,0.00000,0.012736,0.024417,0.017553,0.000000,0.00000,0.015919,0.000000
WASH7P,3.579280,2.329260,4.794250,3.837740,1.472330,3.316980,2.597780,1.540060,2.89535,1.33229,...,4.242780,2.875380,1.06146,1.227010,2.098130,1.919640,1.075360,2.71020,3.295380,2.858990
MIR6859-1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
MIR1302-2HG,0.093825,0.025333,0.000000,0.032159,0.040318,0.071140,0.000000,0.029156,0.00000,0.00000,...,0.000000,0.036142,0.00000,0.050854,0.146248,0.070089,0.036646,0.00000,0.000000,0.049821
FAM138A,0.000000,0.000000,0.046843,0.045693,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,...,0.040545,0.077028,0.00000,0.000000,0.034633,0.000000,0.000000,0.00000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
MT-ND6,4788.560000,5164.360000,3585.510000,7697.900000,3568.410000,6958.790000,4232.780000,4660.220000,5502.63000,13434.70000,...,8543.590000,13099.800000,7783.99000,17519.900000,8798.270000,6874.290000,11061.800000,5241.85000,5626.940000,8785.100000
MT-TE,6.476660,6.676770,6.206630,12.512200,4.554160,12.053700,5.690810,6.586800,9.87533,45.36620,...,20.772700,27.216700,15.41830,50.741300,28.756500,16.713800,15.177700,9.20498,10.371200,28.137900
MT-CYB,28676.500000,26950.900000,20794.900000,23405.400000,14051.500000,19998.100000,24565.300000,24723.700000,24321.90000,20963.20000,...,21229.900000,22350.100000,36174.40000,36777.500000,23341.900000,24296.200000,38732.200000,24934.30000,20992.500000,30362.800000


In [63]:
expr_tpm_FCX.to_csv(f"GTEx_frontal_cortex_gene_TPM.csv")

## SJs

In [ ]:
# Note: this takes a while

expr_sj = pd.read_csv("../GTEx_Analysis_v10_STARv2.7.10a_junctions.gct", sep="\t", header=2)

In [ ]:
expr_sj = expr_sj.set_index("Name", drop=True)
expr_sj = expr_sj.drop("Description", axis=1)

In [ ]:
expr_sj.shape
# >>> expr_sj.shape
# (392955, 19789)

In [ ]:
dtype = np.float64
intron_file = "/mnt/lareaulab/reliscu/data/GENCODE/GRCm39/psix_annotation/intron_file.tab.gz"
intron_table = read_intron_file(intron_file)

In [ ]:
intron_table['intron'] = intron_table['intron'].astype(str)

sj_table = pd.merge(intron_table, expr_sj, # Subset to SJs in annotated intron table
                    left_on="intron", how="left", 
                    right_index=True).fillna(0)
sj_table.shape
# >>> sj_table.shape
# (157406, 19790)

In [ ]:
sj_table_FCX = sj_table.loc[:, sampleinfo_FCX['SAMPID']]

In [ ]:
adata = ad.AnnData(
    X=sj_table_FCX.to_numpy().T,
    var=pd.DataFrame(index=sj_table_FCX.index),
    obs=pd.DataFrame(index=sj_table_FCX.columns),
)

In [ ]:
adata.write_h5ad(f"GTEx_frontal_cortex_SJ_counts.h5ad")